In [1]:
### Some initializations
import numpy as np
import math
import random
import copy
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import lingam
import warnings
import itertools
import loli
from scipy.stats import binomtest
import pickle
from heapq import nlargest
from collections import Counter
from matplotlib.patches import Patch
from sklearn.cluster import KMeans

In [24]:
# Generating coupled data

rs=1
np.random.seed(rs)
size=50000
X=np.zeros((size+1,6))
X[0,:]=[1,0.97,0.99,1,0.97,0.99]


for t in range(size):
    e=np.random.multivariate_normal(np.zeros(6),0.1*np.eye(6))
    x_new=[X[t,0]*0.9+0.1*X[t,1]+e[0],\
           0.25*X[t,0]-0.01*X[t,0]*X[t,2]+0.99*X[t,1]+0.001*X[t,4]**2+e[1],\
           0.01*X[t,0]*X[t,1]+0.9733*X[t,2]+e[2],\
           X[t,3]*0.9+0.1*X[t,4]+e[3],\
           X[t,3]*0.28-X[t,3]*X[t,5]*0.01+0.99*X[t,4]+0.0005*X[t,1]**2+e[4],\
           0.01*X[t,3]*X[t,4]+0.9733*X[t,5]+e[5]]
    X[t+1,:]=x_new


In [25]:

np.random.seed(1)
E=500
n=20
d=6
runs=5
for tar in range(d):
    fpfn=np.zeros((runs,2))
    reportedS={}
    for j in range(runs):
        print('Run ',j+1,'out of ',runs,'target: ',tar)
        dataX=[]
        dataY=[]

        for e in range(E):
                p=j*n+e*n*runs
                dataX.append(X[p:p+n,:])
                dataY.append(X[p+1:p+1+n,tar])
        rs=np.random.randint(low=1,high=1000)
        plausibleS=loli.gauss(dataX,dataY,l=1,alpha=0.1,lam=0,B=500,rs=rs,intercept=False)
        if len(plausibleS)==0:
            supphat=set()
        else:
            supphat=set.intersection(*plausibleS)
        if tuple(supphat) in reportedS:
            reportedS[tuple(supphat)]+=1
        else:
            reportedS[tuple(supphat)]=1

    Lorenz={}
    Lorenz['results']=reportedS
    Lorenz['target']=tar
    Lorenz['runs']=runs
    Lorenz['samplesize']=n
    Lorenz['environments']=E

    with open('LorenzCoupledFixedRsE='+str(E)+'N'+str(n)+'tar'+str(tar)+'.pkl','wb') as f:
        pickle.dump(Lorenz, f)


Run  1 out of  5 target:  0
Run  2 out of  5 target:  0
Run  3 out of  5 target:  0
Run  4 out of  5 target:  0
Run  5 out of  5 target:  0
Run  1 out of  5 target:  1
Run  2 out of  5 target:  1
Run  3 out of  5 target:  1
Run  4 out of  5 target:  1
Run  5 out of  5 target:  1
Run  1 out of  5 target:  2
Run  2 out of  5 target:  2
Run  3 out of  5 target:  2
Run  4 out of  5 target:  2
Run  5 out of  5 target:  2
Run  1 out of  5 target:  3
Run  2 out of  5 target:  3
Run  3 out of  5 target:  3
Run  4 out of  5 target:  3
Run  5 out of  5 target:  3
Run  1 out of  5 target:  4
Run  2 out of  5 target:  4
Run  3 out of  5 target:  4
Run  4 out of  5 target:  4
Run  5 out of  5 target:  4
Run  1 out of  5 target:  5
Run  2 out of  5 target:  5
Run  3 out of  5 target:  5
Run  4 out of  5 target:  5
Run  5 out of  5 target:  5


In [26]:
E=500
n=20
rs=1
for tar in range(6):
        
    with open('LorenzCoupledFixedRsE='+str(E)+'N'+str(n)+'tar'+str(tar)+'.pkl','rb') as f:
        reportedS=pickle.load(f)
    print(tar)
    print(reportedS)
    res=np.zeros(6)
    for key in reportedS['results']:
        for ind in key:
            res[ind]+=reportedS['results'][key]
    print(res)

0
{'results': {(0,): 4, (0, 1): 1}, 'target': 0, 'runs': 5, 'samplesize': 20, 'environments': 500}
[5. 1. 0. 0. 0. 0.]
1
{'results': {(1, 2): 1, (1,): 1, (1, 2, 4, 5): 1, (1, 2, 3, 5): 1, (1, 5): 1}, 'target': 1, 'runs': 5, 'samplesize': 20, 'environments': 500}
[0. 5. 3. 1. 1. 3.]
2
{'results': {(1, 2): 4, (2,): 1}, 'target': 2, 'runs': 5, 'samplesize': 20, 'environments': 500}
[0. 4. 5. 0. 0. 0.]
3
{'results': {(3,): 5}, 'target': 3, 'runs': 5, 'samplesize': 20, 'environments': 500}
[0. 0. 0. 5. 0. 0.]
4
{'results': {(2, 4, 5): 3, (): 2}, 'target': 4, 'runs': 5, 'samplesize': 20, 'environments': 500}
[0. 0. 3. 0. 3. 3.]
5
{'results': {(): 1, (4, 5): 3, (0, 1, 3, 4, 5): 1}, 'target': 5, 'runs': 5, 'samplesize': 20, 'environments': 500}
[1. 1. 0. 1. 4. 4.]


In [28]:
# Generating uncoupled data

rs=1
np.random.seed(rs)
size=50000
X=np.zeros((size+1,6))
X[0,:]=[1,0.97,0.99,1,0.97,0.99]


for t in range(size):
    e=np.random.multivariate_normal(np.zeros(6),0.1*np.eye(6))
    x_new=[X[t,0]*0.9+0.1*X[t,1]+e[0],\
           0.25*X[t,0]-0.01*X[t,0]*X[t,2]+0.99*X[t,1]+e[1],\
           0.01*X[t,0]*X[t,1]+0.9733*X[t,2]+e[2],\
           X[t,3]*0.9+0.1*X[t,4]+e[3],\
           X[t,3]*0.28-X[t,3]*X[t,5]*0.01+0.99*X[t,4]+e[4],\
           0.01*X[t,3]*X[t,4]+0.9733*X[t,5]+e[5]]
    X[t+1,:]=x_new


In [35]:
E=500
n=25
d=6
runs=4
np.random.seed(1)
for tar in range(d):
    fpfn=np.zeros((runs,2))
    reportedS={}
    for j in range(runs):
        print('Run ',j+1,'out of ',runs,'target: ',tar)
        dataX=[]
        dataY=[]

        for e in range(E):
                p=j*n+e*n*runs
                dataX.append(X[p:p+n,:])
                dataY.append(X[p+1:p+1+n,tar])
        rs=np.random.randint(low=1,high=1000)
        plausibleS=loli.gauss(dataX,dataY,l=1,alpha=0.1,lam=0,B=500,rs=rs)
        if len(plausibleS)==0:
            supphat=set()
        else:
            supphat=set.intersection(*plausibleS)
        if tuple(supphat) in reportedS:
            reportedS[tuple(supphat)]+=1
        else:
            reportedS[tuple(supphat)]=1

    Lorenz={}
    Lorenz['results']=reportedS
    Lorenz['target']=tar
    Lorenz['runs']=runs
    Lorenz['samplesize']=n
    Lorenz['environments']=E

    with open('LorenzUncoupledFixedRsE='+str(E)+'N'+str(n)+'tar'+str(tar)+'.pkl','wb') as f:
        pickle.dump(Lorenz, f)

Run  1 out of  4 target:  0
Run  2 out of  4 target:  0
Run  3 out of  4 target:  0
Run  4 out of  4 target:  0
Run  1 out of  4 target:  1
Run  2 out of  4 target:  1
Run  3 out of  4 target:  1
Run  4 out of  4 target:  1
Run  1 out of  4 target:  2
Run  2 out of  4 target:  2
Run  3 out of  4 target:  2
Run  4 out of  4 target:  2
Run  1 out of  4 target:  3
Run  2 out of  4 target:  3
Run  3 out of  4 target:  3
Run  4 out of  4 target:  3
Run  1 out of  4 target:  4
Run  2 out of  4 target:  4
Run  3 out of  4 target:  4
Run  4 out of  4 target:  4
Run  1 out of  4 target:  5
Run  2 out of  4 target:  5
Run  3 out of  4 target:  5
Run  4 out of  4 target:  5


In [36]:
E=500
n=25
rs=1
for tar in range(6):
        
    with open('LorenzUncoupledFixedRsE='+str(E)+'N'+str(n)+'tar'+str(tar)+'.pkl','rb') as f:
        reportedS=pickle.load(f)
    print(tar)
    print(reportedS)
    res=np.zeros(6)
    for key in reportedS['results']:
        for ind in key:
            res[ind]+=reportedS['results'][key]
    print(res)

0
{'results': {(0,): 4}, 'target': 0, 'runs': 4, 'samplesize': 25, 'environments': 500}
[4. 0. 0. 0. 0. 0.]
1
{'results': {(1,): 2, (1, 2): 1, (1, 2, 4): 1}, 'target': 1, 'runs': 4, 'samplesize': 25, 'environments': 500}
[0. 4. 2. 0. 1. 0.]
2
{'results': {(1, 2): 3, (0, 1, 2, 4): 1}, 'target': 2, 'runs': 4, 'samplesize': 25, 'environments': 500}
[1. 4. 4. 0. 1. 0.]
3
{'results': {(3,): 3, (3, 4): 1}, 'target': 3, 'runs': 4, 'samplesize': 25, 'environments': 500}
[0. 0. 0. 4. 1. 0.]
4
{'results': {(): 2, (3, 4): 1, (4, 5): 1}, 'target': 4, 'runs': 4, 'samplesize': 25, 'environments': 500}
[0. 0. 0. 1. 2. 1.]
5
{'results': {(4, 5): 2, (2, 3, 4, 5): 1, (): 1}, 'target': 5, 'runs': 4, 'samplesize': 25, 'environments': 500}
[0. 0. 1. 1. 3. 3.]
